<h2><a href="https://leetcode.com/problems/customers-who-bought-products-a-and-b-but-not-c">1536. Customers Who Bought Products A and B but Not C</a></h2><h3>Medium</h3><hr><p>Table: <code>Customers</code></p>

<pre>
+---------------------+---------+
| Column Name         | Type    |
+---------------------+---------+
| customer_id         | int     |
| customer_name       | varchar |
+---------------------+---------+
customer_id is the column with unique values for this table.
customer_name is the name of the customer.</pre>

<p>&nbsp;</p>

<p>Table: <code>Orders</code></p>

<pre>
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| order_id      | int     |
| customer_id   | int     |
| product_name  | varchar |
+---------------+---------+
order_id is the column with unique values for this table.
customer_id is the id of the customer who bought the product &quot;product_name&quot;.
</pre>

<p>&nbsp;</p>

<p>Write a solution&nbsp;to report the customer_id and customer_name of customers who bought products <strong>&quot;A&quot;</strong>, <strong>&quot;B&quot;</strong> but did not buy the product <strong>&quot;C&quot;</strong> since we want to recommend them to purchase this product.</p>

<p>Return the result table <strong>ordered</strong> by <code>customer_id</code>.</p>

<p>The&nbsp;result format is in the following example.</p>

<p>&nbsp;</p>
<p><strong class="example">Example 1:</strong></p>

<pre>
<strong>Input:</strong> 
Customers table:
+-------------+---------------+
| customer_id | customer_name |
+-------------+---------------+
| 1           | Daniel        |
| 2           | Diana         |
| 3           | Elizabeth     |
| 4           | Jhon          |
+-------------+---------------+
Orders table:
+------------+--------------+---------------+
| order_id   | customer_id  | product_name  |
+------------+--------------+---------------+
| 10         |     1        |     A         |
| 20         |     1        |     B         |
| 30         |     1        |     D         |
| 40         |     1        |     C         |
| 50         |     2        |     A         |
| 60         |     3        |     A         |
| 70         |     3        |     B         |
| 80         |     3        |     D         |
| 90         |     4        |     C         |
+------------+--------------+---------------+
<strong>Output:</strong> 
+-------------+---------------+
| customer_id | customer_name |
+-------------+---------------+
| 3           | Elizabeth     |
+-------------+---------------+
<strong>Explanation:</strong> Only the customer_id with id 3 bought the product A and B but not the product C.
</pre>

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col

# Initialize Spark Session
spark = SparkSession.builder.master("local").appName("CustomersWhoBoughtAandBNotC").getOrCreate()

# Define Schema for Customers Table
customers_schema = StructType([
    StructField("customer_id", IntegerType(), False),
    StructField("customer_name", StringType(), False)
])

# Define Schema for Orders Table
orders_schema = StructType([
    StructField("order_id", IntegerType(), False),
    StructField("customer_id", IntegerType(), False),
    StructField("product_name", StringType(), False)
])

# Hardcoded Customers Data
customers_data = [
    (1, "Daniel"),
    (2, "Diana"),
    (3, "Elizabeth"),
    (4, "Jhon")
]

# Hardcoded Orders Data
orders_data = [
    (10, 1, "A"),
    (20, 1, "B"),
    (30, 1, "D"),
    (40, 1, "C"),
    (50, 2, "A"),
    (60, 3, "A"),
    (70, 3, "B"),
    (80, 3, "D"),
    (90, 4, "C")
]

# Create DataFrames
customers_df = spark.createDataFrame(customers_data, schema=customers_schema)
orders_df = spark.createDataFrame(orders_data, schema=orders_schema)

# Display DataFrames
print("Customers Table:")
customers_df.show()

print("Orders Table:")
orders_df.show()

Customers Table:
+-----------+-------------+
|customer_id|customer_name|
+-----------+-------------+
|          1|       Daniel|
|          2|        Diana|
|          3|    Elizabeth|
|          4|         Jhon|
+-----------+-------------+

Orders Table:
+--------+-----------+------------+
|order_id|customer_id|product_name|
+--------+-----------+------------+
|      10|          1|           A|
|      20|          1|           B|
|      30|          1|           D|
|      40|          1|           C|
|      50|          2|           A|
|      60|          3|           A|
|      70|          3|           B|
|      80|          3|           D|
|      90|          4|           C|
+--------+-----------+------------+



In [7]:
# Create temporary views for both DataFrames
customers_df.createOrReplaceTempView("Customers")
orders_df.createOrReplaceTempView("Orders")

In [14]:
spark.sql("""
    SELECT temp.customer_id,
          c.customer_name
        FROM  (SELECT *,
                CASE 
                  WHEN product_name = 'A' THEN 1
                  ELSE 0
                END as IS_A,
                CASE 
                  WHEN product_name = 'B' THEN 1
                  ELSE 0
                END as IS_B,
                CASE 
                  WHEN product_name = 'C' THEN 1
                  ELSE 0
                END as IS_C
                FROM Orders) temp
          INNER JOIN Customers c ON temp.customer_id = c.customer_id
        GROUP BY temp.customer_id, c.customer_name
        HAVING (CASE
            WHEN (SUM(IS_A) > 0) AND SUM(IS_B) > 0 AND SUM(IS_C) = 0 THEN 'Y'
            ELSE 'N'
          END) = 'Y'
""").show()

+-----------+-------------+
|customer_id|customer_name|
+-----------+-------------+
|          3|    Elizabeth|
+-----------+-------------+



In [15]:
spark.stop()